In [23]:
import pandas as pd
import numpy as np
import re
import pyodbc
from sqlalchemy import create_engine
import urllib
import datetime as dt
from pathlib3x import Path
import easygui

In [24]:

def selecionarArquivo(*pasta):
    """
    Selecionar um arquivo para servir de carga no SAP
    Returns:
        String: retorna um caminho em forma de string (texto)
    """
    global local_arquivo
    local_arquivo = ''

    if not pasta:
        arquivo = easygui.fileopenbox()
        local_arquivo = Path(arquivo)   
    else:
        print(f"o arquivo da pasta: {pasta[0]}")
        local_arquivo = pasta[0]
        
    return local_arquivo

###### Selecionar a pasta do arquivo do mês ########

localArquivo = selecionarArquivo()

In [25]:
localArquivo

WindowsPath('W:/04 - Diretoria Financeira/Contas-a-pagar/NESA/Financeiro/ANO 2023/APLICAÇÕES FINANCEIRAS/Carga_SQL/SQL_base_Aplicacao_23-10-2023.xlsx')

In [26]:
"""
Banco de dados: 

[DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS]
[DW_SFCRI].[TESOURARIA].[TITULOS]
[DW_SFCRI].[TESOURARIA].[BANCOS]


sheets: 

TITULOS, APLICACAO_FUNDOS, BANCOS

"""


'\nBanco de dados: \n\n[DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS]\n[DW_SFCRI].[TESOURARIA].[TITULOS]\n[DW_SFCRI].[TESOURARIA].[BANCOS]\n\n\nsheets: \n\nTITULOS, APLICACAO_FUNDOS, BANCOS\n\n'

In [27]:
df_bancos = pd.read_excel(localArquivo, sheet_name='BANCOS')
df_titulos = pd.read_excel(localArquivo, sheet_name='TITULOS')
df_aplicacao = pd.read_excel(localArquivo, sheet_name='APLICACAO_FUNDOS')

In [28]:
df_titulos['LIQUIDEZ'] = df_titulos['LIQUIDEZ'].astype(str)
df_titulos

,TIPO,EMISSOR,APLICACAO,NUM_SAP_TRM,DATA_APLICACAO,DATA_VENCIMENTO,LIQUIDEZ,CP_LP,TIPO_APLICACAO,DIAS_PARA_LIQUIDEZ,TAXA_CONTRATADA
0,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_BNDES,100000000042,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
1,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_AGENTES,100000000043,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
2,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_OM,100000000045,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
3,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_ONS,100000000048,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
4,FUNDOS,CEF FUNDO XINGU,CEF_FXINGU_IBAMA,100000000046,NaT,2042-01-31,Diária,NaN,Fundo,0,1.13
...,...,...,...,...,...,...,...,...,...,...,...
213,APLICACOES,ITAU,ITAU_LF_2605_01,LFSC190006N,2021-05-26,2029-01-08,Diária,LP,LF,0,1.35
214,APLICACOES,ITAU,ITAU_LF_2605_02,LFSC1900072,2021-05-26,2029-01-08,Diária,LP,LF,0,1.35
215,APLICACOES,ITAU,ITAU_LF_2605_03,LFSC190006P,2021-05-26,2029-07-16,Diária,LP,LF,0,1.35
216,APLICACOES,ITAU,ITAU_LF_2605_04,LFSC190006S,2021-05-26,2029-07-16,Diária,LP,LF,0,1.35


In [29]:
df_titulos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   TIPO                218 non-null    object        
 1   EMISSOR             218 non-null    object        
 2   APLICACAO           218 non-null    object        
 3   NUM_SAP_TRM         218 non-null    object        
 4   DATA_APLICACAO      212 non-null    datetime64[ns]
 5   DATA_VENCIMENTO     217 non-null    datetime64[ns]
 6   LIQUIDEZ            218 non-null    object        
 7   CP_LP               212 non-null    object        
 8   TIPO_APLICACAO      218 non-null    object        
 9   DIAS_PARA_LIQUIDEZ  218 non-null    int64         
 10  TAXA_CONTRATADA     218 non-null    float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 18.9+ KB


In [30]:
df_bancos

,CODIGO,NATUREZA_BANCO,BANCO,GRUPO_POLITICA,DATA_PL,S&P,FITCH,MOODYS,INDICE_BASILEIA,PL_RS_MIL,PERC_EXPOSICAO_POLITICA,LIMITE _BANCO
0,DFs BACEN,Público,BACEN,Grupo 1,2022-12-31,NaN,NaN,NaN,0.0000,6.101200e+10,0.08,4.880960e+09
1,1000080329,Público,BB,Grupo 1,2022-12-31,NaN,NaN,NaN,0.1619,1.592595e+11,0.08,1.274076e+10
2,1000080738,Público,CEF,Grupo 1,2022-12-31,AAA,AA,AA,0.1763,8.714610e+10,0.08,6.971688e+09
3,1000081593,Público,BNB,Grupo 1,2022-12-31,NaN,NaN,NaN,0.1337,9.496301e+09,0.08,7.597041e+08
4,1000080288,Público,BRB,Grupo 1,2022-12-31,NaN,AA,NaN,0.1491,2.630660e+09,0.08,2.104528e+08
5,1000080099,Privado,ITAU,Grupo 2,2022-12-31,AAA,AAA,A,0.1498,1.750281e+11,0.07,1.225196e+10
6,1000080075,Privado,BRADESCO,Grupo 2,2022-12-31,NaN,AAA,NaN,0.1509,1.562381e+11,0.07,1.093666e+10
7,1000080336,Privado,BTG,Grupo 2,2022-12-31,AA,AA,AA,0.1552,4.854097e+10,0.06,2.912458e+09
8,1000080109,Privado,SAFRA,Grupo 2,2022-12-31,AAA,NaN,NaN,0.1841,2.389896e+10,0.07,1.672927e+09
9,1000082475,Privado,XP,Grupo 3,2022-12-31,NaN,AA,NaN,0.2035,9.536003e+09,0.05,4.768002e+08


In [31]:
# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que deseja limpar

df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].astype(str)
df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].str.strip()
df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].apply(lambda x: float(x) if x.strip() else 0.0)

In [32]:
df_aplicacao

,TIPO,APLICACAO,DATA,CDI_100%,FATOR_DIARIO,VALOR_APLICACAO,RESGATE,VALOR_COTA,QTD_COTAS,GANHO_DIARIO,SALDO_BRUTO
0,FUNDOS,CEF_FXINGU_BNDES,2020-10-31,1.000075,1.000084,NaN,NaN,2.037246,2.422558e+07,0,4.935346e+07
1,FUNDOS,CEF_FXINGU_BNDES,2020-11-03,1.000075,1.000084,NaN,NaN,2.037412,2.422558e+07,4026.776788,4.935748e+07
2,FUNDOS,CEF_FXINGU_BNDES,2020-11-04,1.000075,1.000084,NaN,NaN,2.037537,2.422558e+07,3038.371864,4.936052e+07
3,FUNDOS,CEF_FXINGU_BNDES,2020-11-05,1.000075,1.000084,NaN,NaN,2.037558,2.422558e+07,490.567934,4.936101e+07
4,FUNDOS,CEF_FXINGU_BNDES,2020-11-06,1.000075,1.000084,NaN,NaN,2.037627,2.422558e+07,1681.497298,4.936269e+07
...,...,...,...,...,...,...,...,...,...,...,...
125807,APLICACOES,ITAU_261023_80,2023-11-20,1.000473,1.000378,NaN,NaN,1.005688,3.399999e+07,12928.081227,3.419340e+07
125808,APLICACOES,ITAU_261023_80,2023-11-21,1.000473,1.000378,NaN,NaN,1.006069,3.399999e+07,12932.971015,3.420634e+07
125809,APLICACOES,ITAU_261023_80,2023-11-22,1.000473,1.000378,NaN,NaN,1.006449,3.399999e+07,12937.862653,3.421927e+07
125810,APLICACOES,ITAU_261023_80,2023-11-23,1.000473,1.000378,NaN,NaN,1.006830,3.399999e+07,12942.756141,3.423222e+07


In [33]:

#df_aplicacao['GANHO_DIARIO'] = df_aplicacao['GANHO_DIARIO'].str.replace(',', '.')
#df_aplicacao['GANHO_DIARIO'] = df_aplicacao['GANHO_DIARIO'].str.replace(',', '.')
#df_aplicacao['GANHO_DIARIO'] = pd.to_numeric(df_aplicacao['GANHO_DIARIO'], errors='coerce')

df_aplicacao['GANHO_DIARIO'] = df_aplicacao['GANHO_DIARIO'].astype(str).apply(lambda x: x.replace(",", "."))
df_aplicacao['GANHO_DIARIO'] = df_aplicacao['GANHO_DIARIO'].replace('.', '0.0')
df_aplicacao['GANHO_DIARIO'] = pd.to_numeric(df_aplicacao['GANHO_DIARIO'].str.replace(',', '', regex=True))  






In [34]:
df_aplicacao['GANHO_DIARIO']

0             0.000000
1          4026.776788
2          3038.371864
3           490.567934
4          1681.497298
              ...     
125807    12928.081227
125808    12932.971015
125809    12937.862653
125810    12942.756141
125811    12947.651479
Name: GANHO_DIARIO, Length: 125812, dtype: float64

In [35]:
df_aplicacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125812 entries, 0 to 125811
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   TIPO             125812 non-null  object        
 1   APLICACAO        125812 non-null  object        
 2   DATA             125812 non-null  datetime64[ns]
 3   CDI_100%         125812 non-null  float64       
 4   FATOR_DIARIO     125812 non-null  float64       
 5   VALOR_APLICACAO  278 non-null     float64       
 6   RESGATE          480 non-null     float64       
 7   VALOR_COTA       125812 non-null  float64       
 8   QTD_COTAS        125812 non-null  float64       
 9   GANHO_DIARIO     125812 non-null  float64       
 10  SALDO_BRUTO      125812 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(2)
memory usage: 10.6+ MB


#### carga SQL

In [36]:
conexao = pyodbc.connect('Driver={SQL Server};'
                      'Server=CURIAIA1-10-3\FINANCEIRO_PRD;'
                      'Database=DW_SFCRI;'
                      'Trusted_Connection=yes;')

cursor = conexao.cursor() # Cria o cursor para manipular o banco de dados
truncate_APLICACAO_FUNDOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS] """
# truncate_APLICACAO_FUNDOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[APLICACAO_FUNDOS_2] """
truncate_BANCOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[BANCOS] """
truncate_TITULOS = """ TRUNCATE TABLE [DW_SFCRI].[TESOURARIA].[TITULOS] """

In [37]:
cursor.execute(truncate_APLICACAO_FUNDOS) # Executa a Limpeza dos dados 
cursor.execute(truncate_BANCOS) # Executa a Limpeza dos dados 
cursor.execute(truncate_TITULOS) # Executa a Limpeza dos dados 
cursor.commit() # Confirma

In [38]:
pst = urllib.parse.quote_plus('Driver={SQL Server};'
                                'Server=CURIAIA1-10-3\FINANCEIRO_PRD;'
                                'Database=DW_SFCRI;'
                                'Trusted_Connection=yes;')
engine = create_engine(f'mssql+pyodbc:///?odbc_connect={pst}')

In [39]:
df_titulos.to_sql('TITULOS', con=engine, if_exists='append', index=False, schema="TESOURARIA")
df_bancos.to_sql('BANCOS', con=engine, if_exists='append', index=False, schema="TESOURARIA")
print('Começando a carga de aplicação')
# df_aplicacao.to_sql('APLICACAO_FUNDOS_2', con=engine, if_exists='replace', index=False, schema="TESOURARIA")
df_aplicacao.to_sql('APLICACAO_FUNDOS', con=engine, if_exists='replace', index=False, schema="TESOURARIA")
print('Fim da carga de aplicação')


Começando a carga de aplicação
Fim da carga de aplicação


In [40]:
import win32com.client as win32

# criar a integração com o outlook
outlook = win32.Dispatch('outlook.application')

# criar um email
email = outlook.CreateItem(0)

# configurar as informações do seu e-mail
email.To = "contasapagar@norteenergiasa.com.br;CleitonJunior@norteenergiasa.com.br"
email.Subject = "E-mail automático do Python"
email.HTMLBody = f"""

<p>Carga de controle de aplicações carregada no SQL</p>
<p>Dados com movimentações 26/10/2023. </p>
<p>Abs,</p>
<p>Código Python</p>
"""

#anexo = "W:\\04 - Diretoria Financeira\\Contas-a-pagar\\NESA\\Financeiro\\ANO 2023\\APLICAÇÕES FINANCEIRAS\\Controle Aplicações.xlsx"
#email.Attachments.Add(anexo)

email.Send()
print("Email Enviado")


Email Enviado


#### Testes tipos

In [41]:
# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que deseja limpar

# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].astype(str)
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].str.strip()

In [42]:
# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que você deseja converter
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].apply(lambda x: float(x) if x.strip() else 0.0)


In [43]:

# Suponha que 'df_aplicacao' seja o seu DataFrame e 'VALOR_APLICACAO' seja a coluna que você deseja converter
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].apply(lambda x: float(x.strip()) if isinstance(x, str) and x.strip() else 0.0)


In [44]:
# df_aplicacao['VALOR_APLICACAO'] = df_aplicacao['VALOR_APLICACAO'].astype(float)